In [44]:
!pip install torch-optimizer

In [24]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, Subset
from torchvision.transforms import ToTensor
import torch.cuda.amp as amp
from torchvision import transforms, models

from skimage.metrics import structural_similarity as ssim
from skimage.metrics import peak_signal_noise_ratio as psnr
from sklearn.model_selection import train_test_split

import numpy as np
import matplotlib.pyplot as plt
import os
from torch_optimizer import RAdam 
import torchmetrics
from torchmetrics.image import StructuralSimilarityIndexMeasure
from torchmetrics.image import PeakSignalNoiseRatio

In [5]:
torch.manual_seed(42)
torch.cuda.manual_seed_all(42)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [6]:
class RealLensingDataset(Dataset):
    def __init__(self, lr_paths, hr_paths, augment=False, denoise=False):
        """
        Args:
            lr_paths: List of paths to LR .npy files.
            hr_paths: List of paths to HR .npy files.
            augment: If True, apply random flips and rotations.
            denoise: If True, apply Gaussian blur to reduce noise.
        """
        self.lr_paths = lr_paths
        self.hr_paths = hr_paths
        self.augment = augment
        self.denoise = denoise
        self.denoise_transform = transforms.GaussianBlur(kernel_size=3, sigma=1.0) if denoise else None
        self.augment_transforms = transforms.Compose([
            transforms.RandomHorizontalFlip(p=0.5),
            transforms.RandomVerticalFlip(p=0.5),
            transforms.RandomRotation(degrees=(0, 360), interpolation=transforms.InterpolationMode.BILINEAR)
        ]) if augment else None
    
    def __len__(self):
        return len(self.lr_paths)
    
    def __getitem__(self, idx):
        lr = np.load(self.lr_paths[idx]).astype(np.float32)
        hr = np.load(self.hr_paths[idx]).astype(np.float32)
        
        # Normalize to [0, 1]
        lr_min, lr_max = lr.min(), lr.max()
        if lr_max > lr_min:
            lr = (lr - lr_min) / (lr_max - lr_min)
        else:
            lr = np.zeros_like(lr)
        hr_min, hr_max = hr.min(), hr.max()
        if hr_max > hr_min:
            hr = (hr - hr_min) / (hr_max - hr_min)
        else:
            hr = np.zeros_like(hr)
        
        lr = torch.from_numpy(lr)
        hr = torch.from_numpy(hr)
        
        # Apply denoising if enabled
        if self.denoise and self.denoise_transform:
            lr = self.denoise_transform(lr)
            hr = self.denoise_transform(hr)
        
        # Apply augmentation if enabled
        if self.augment and self.augment_transforms:
            seed = torch.randint(0, 2**32, (1,)).item()
            torch.manual_seed(seed)
            lr = self.augment_transforms(lr)
            torch.manual_seed(seed)
            hr = self.augment_transforms(hr)
        
        return lr, hr

In [7]:
lr_dir = "/kaggle/input/dataset-image-super-resolution-3b/Dataset/LR"
hr_dir = "/kaggle/input/dataset-image-super-resolution-3b/Dataset/HR"

In [8]:
lr_files = sorted([os.path.join(lr_dir, f) for f in os.listdir(lr_dir) if f.endswith('.npy')])
hr_files = sorted([os.path.join(hr_dir, f) for f in os.listdir(hr_dir) if f.endswith('.npy')])
assert len(lr_files) == len(hr_files) == 300, "LR/HR mismatch"

In [9]:
indices = list(range(300))
train_idx, temp_idx = train_test_split(indices, test_size=60, random_state=42)
val_idx, test_idx = train_test_split(temp_idx, test_size=30, random_state=42)

In [10]:
# Datasets for different stages
dataset_noaug = RealLensingDataset(lr_files, hr_files, augment=False, denoise=False)
dataset_aug = RealLensingDataset(lr_files, hr_files, augment=True, denoise=False)
dataset_aug_denoise = RealLensingDataset(lr_files, hr_files, augment=True, denoise=True)

In [11]:
train_dataset_noaug = Subset(dataset_noaug, train_idx)
val_dataset = Subset(dataset_noaug, val_idx)  # No aug for val/test
test_dataset = Subset(dataset_noaug, test_idx)

In [12]:
train_dataset_aug = Subset(dataset_aug, train_idx)
train_dataset_aug_denoise = Subset(dataset_aug_denoise, train_idx)

In [17]:
train_loader_noaug = DataLoader(train_dataset_noaug, batch_size=8, shuffle=True, num_workers=8, pin_memory=True, prefetch_factor=2)
train_loader_aug = DataLoader(train_dataset_aug, batch_size=8, shuffle=True, num_workers=8, pin_memory=True, prefetch_factor=2)
train_loader_aug_denoise = DataLoader(train_dataset_aug_denoise, batch_size=8, shuffle=True, num_workers=8, pin_memory=True, prefetch_factor=2)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False, num_workers=8, pin_memory=True, prefetch_factor=2)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False, num_workers=8, pin_memory=True, prefetch_factor=2)

In [18]:
class PerceptualLoss(nn.Module):
    def __init__(self):
        super(PerceptualLoss, self).__init__()
        vgg = models.vgg19(weights=models.VGG19_Weights.IMAGENET1K_V1).features[:29].eval()
        self.vgg = vgg.to("cuda")
        for param in self.vgg.parameters():
            param.requires_grad = False
        self.mse = nn.MSELoss()
    
    def forward(self, sr, hr):
        sr = sr.repeat(1, 3, 1, 1)
        hr = hr.repeat(1, 3, 1, 1)
        sr_features = self.vgg(sr)
        hr_features = self.vgg(hr)
        return self.mse(sr_features, hr_features)

In [19]:
class ResBlock(nn.Module):
    def __init__(self, channels=64):
        super(ResBlock, self).__init__()
        self.conv1 = nn.Conv2d(channels, channels, 3, padding=1, bias=False)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(channels, channels, 3, padding=1, bias=False)
        nn.init.xavier_uniform_(self.conv1.weight)
        nn.init.xavier_uniform_(self.conv2.weight)
    
    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.relu(out)
        out = self.conv2(out)
        return out + 0.3 * residual  
        
class EDSR(nn.Module):
    def __init__(self, scale_factor=2, num_blocks=8, channels=64):  # 8 blocks
        super(EDSR, self).__init__()
        self.input_conv = nn.Conv2d(1, channels, 3, padding=1, bias=False)
        self.res_blocks = nn.Sequential(*[ResBlock(channels) for _ in range(num_blocks)])
        self.mid_conv = nn.Conv2d(channels, channels, 3, padding=1, bias=False)
        self.upsample = nn.Sequential(
            nn.Conv2d(channels, channels*4, 3, padding=1, bias=False),
            nn.PixelShuffle(2)
        )
        self.output_conv = nn.Conv2d(channels, 1, 3, padding=1, bias=False)
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.xavier_uniform_(m.weight)
    
    def forward(self, x):
        x = self.input_conv(x)
        residual = x
        x = self.res_blocks(x)
        x = self.mid_conv(x) + 0.3 * residual
        x = self.upsample(x)
        x = self.output_conv(x)
        return torch.clamp(x, 0, 1)

In [20]:
#Args:
#model: EDSR model to fine-tune
#train_loader: DataLoader for training data
#val_loader: DataLoader for validation dat
#epochs: Number of epochs to train
#lr: Learning rate
#use_perceptual: If True, use perceptual loss
#use_radam: If True, use RAdam optimizer
#use_scheduler: If True, use CosineAnnealingWarmRestarts
#stage_name: Name of the stage for logging

def finetune(model, train_loader, val_loader, epochs, lr=5e-4, use_perceptual=False, use_radam=False, use_scheduler=False, stage_name="Stage"):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    
    # Loss functions
    l1_criterion = nn.L1Loss()
    mse_criterion = nn.MSELoss()
    perceptual_criterion = PerceptualLoss() if use_perceptual else None
    
    # Optimizer
    if use_radam:
        optimizer = RAdam(model.parameters(), lr=lr, weight_decay=1e-4)
    else:
        optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=1e-4)
    
    # Scheduler
    scheduler = optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=20, T_mult=1) if use_scheduler else None
    
    scaler = torch.amp.GradScaler('cuda')
    train_losses, val_losses = [], []
    best_val_loss = float('inf')
    
    # Debug output
    print(f"\n=== {stage_name} ===")
    print(f"Model on device: {next(model.parameters()).device}")
    print(f"Initial VRAM usage: {torch.cuda.memory_allocated() / 1024**2:.2f} MB")
    lr, hr = next(iter(train_loader))
    print(f"LR shape: {lr.shape}, HR shape: {hr.shape}")
    with torch.no_grad():
        sr = model(lr.to(device))
        print(f"SR shape: {sr.shape}, SR min: {sr.min().item():.4f}, max: {sr.max().item():.4f}")
    
    # Training loop
    for epoch in range(epochs):
        model.train()
        train_loss = 0
        for lr, hr in train_loader:
            lr, hr = lr.to(device), hr.to(device)
            optimizer.zero_grad()
            with torch.amp.autocast('cuda'):
                sr = model(lr)
                l1_loss = l1_criterion(sr, hr)
                loss = l1_loss
                if use_perceptual:
                    mse_loss = mse_criterion(sr, hr)
                    perc_loss = perceptual_criterion(sr, hr)
                    loss = l1_loss + 0.1 * mse_loss + (0.7 if stage_name == "Stage 5" else 0.5) * perc_loss
            scaler.scale(loss).backward()
            scaler.unscale_(optimizer)
            nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            scaler.step(optimizer)
            scaler.update()
            train_loss += l1_loss.item() * lr.size(0)
        train_loss /= len(train_loader.dataset)
        train_losses.append(train_loss)
        
        # Validation
        model.eval()
        val_loss = 0
        with torch.no_grad():
            for lr, hr in val_loader:
                lr, hr = lr.to(device), hr.to(device)
                with torch.amp.autocast('cuda'):
                    sr = model(lr)
                    loss = l1_criterion(sr, hr)
                val_loss += loss.item() * lr.size(0)
        val_loss /= len(val_loader.dataset)
        val_losses.append(val_loss)
        
        # Scheduler step
        if scheduler:
            scheduler.step()
        
        print(f"Epoch {epoch+1}/{epochs}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}")
        
        if (epoch + 1) % 5 == 0:
            with torch.no_grad():
                sr = model(lr.to(device))
                print(f"Epoch {epoch+1} SR min: {sr.min().item():.4f}, max: {sr.max().item():.4f}")
        
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            torch.save(model.state_dict(), f"edsr_finetuned_best_{stage_name}.pth")
    
    plt.figure(figsize=(10, 5))
    plt.plot(range(1, epochs+1), train_losses, label="Train Loss")
    plt.plot(range(1, epochs+1), val_losses, label="Val Loss")
    plt.xlabel("Epoch")
    plt.ylabel("L1 Loss")
    plt.title(f"{stage_name} Fine-Tuning Loss Curves")
    plt.legend()
    plt.grid()
    plt.savefig(f"{stage_name}_loss_curves.png")
    plt.close()
    
    return model

In [21]:
def evaluate_split(model, loader, split_name, num_vis=3):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    model.eval()
    
    mse_metric = torchmetrics.MeanSquaredError().to(device)
    ssim_metric = torchmetrics.StructuralSimilarityIndexMeasure(data_range=1.0).to(device)
    psnr_metric = torchmetrics.PeakSignalNoiseRatio(data_range=1.0).to(device)
    
    vis_lr, vis_hr, vis_sr = [], [], []
    with torch.no_grad():
        for i, (lr, hr) in enumerate(loader):
            lr, hr = lr.to(device), hr.to(device)
            sr = model(lr)
            mse_metric.update(sr, hr)
            ssim_metric.update(sr, hr)
            psnr_metric.update(sr, hr)
            if i < num_vis:
                vis_lr.append(lr.cpu())
                vis_hr.append(hr.cpu())
                vis_sr.append(sr.cpu())
    
    mse = mse_metric.compute()
    ssim = ssim_metric.compute()
    psnr = psnr_metric.compute()
    print(f"{split_name} MSE: {mse:.4f}, SSIM: {ssim:.4f}, PSNR: {psnr:.4f}")
    
    for i in range(num_vis):
        fig, axes = plt.subplots(1, 3, figsize=(15, 5))  # 1 row, 3 columns
        axes[0].imshow(vis_lr[i][0, 0], cmap="gray")
        axes[0].set_title(f"{split_name} LR")
        axes[1].imshow(vis_sr[i][0, 0], cmap="gray")
        axes[1].set_title(f"{split_name} SR")
        axes[2].imshow(vis_hr[i][0, 0], cmap="gray")
        axes[2].set_title(f"{split_name} HR")
        for ax in axes:
            ax.axis("off")
        plt.tight_layout()
        plt.savefig(f"sr_visuals_{split_name.lower()}_sidebyside_{i+1}.png")
        plt.close()

In [22]:
# Task 1 model
model = EDSR(scale_factor=2, num_blocks=8, channels=64)
model.load_state_dict(torch.load("/kaggle/input/edsr-best-pth/edsr_best.pth", weights_only=True))

<All keys matched successfully>

In [29]:
# Stage 1: Basic Fine-Tuning (No Augmentation, L1 Loss)
# - Start with the Task 1 model, fine-tune on real data with minimal changes.
# - No augmentation to establish a baseline.
# - Expected: PSNR ~28-29 dB, SSIM ~0.8 due to real data noise.
model = finetune(model, train_loader_noaug, val_loader, epochs=50, lr=5e-4, use_perceptual=False, use_radam=False, use_scheduler=False, stage_name="Stage1")
evaluate_split(model, train_loader_noaug, "Train_Stage1", num_vis=3)
evaluate_split(model, val_loader, "Val_Stage1", num_vis=3)
evaluate_split(model, test_loader, "Test_Stage1", num_vis=3)
torch.save(model.state_dict(), "edsr_finetuned_stage1.pth")
print("Final fine-tuned model saved as edsr_finetuned_stage1.pth")


=== Stage1 ===
Model on device: cuda:0
Initial VRAM usage: 13.75 MB
LR shape: torch.Size([8, 1, 64, 64]), HR shape: torch.Size([8, 1, 128, 128])
SR shape: torch.Size([8, 1, 128, 128]), SR min: 0.0032, max: 0.2334
Epoch 1/50, Train Loss: 0.0172, Val Loss: 0.0166
Epoch 2/50, Train Loss: 0.0172, Val Loss: 0.0162
Epoch 3/50, Train Loss: 0.0174, Val Loss: 0.0164
Epoch 4/50, Train Loss: 0.0175, Val Loss: 0.0162
Epoch 5/50, Train Loss: 0.0176, Val Loss: 0.0180
Epoch 5 SR min: 0.0009, max: 0.1845
Epoch 6/50, Train Loss: 0.0172, Val Loss: 0.0161
Epoch 7/50, Train Loss: 0.0176, Val Loss: 0.0161
Epoch 8/50, Train Loss: 0.0172, Val Loss: 0.0167
Epoch 9/50, Train Loss: 0.0173, Val Loss: 0.0168
Epoch 10/50, Train Loss: 0.0173, Val Loss: 0.0163
Epoch 10 SR min: 0.0011, max: 0.2311
Epoch 11/50, Train Loss: 0.0172, Val Loss: 0.0161
Epoch 12/50, Train Loss: 0.0176, Val Loss: 0.0169
Epoch 13/50, Train Loss: 0.0174, Val Loss: 0.0171
Epoch 14/50, Train Loss: 0.0174, Val Loss: 0.0170
Epoch 15/50, Train Los

/usr/local/lib/python3.10/dist-packages/torchmetrics/utilities/prints.py:62: FutureWarning: Importing `StructuralSimilarityIndexMeasure` from `torchmetrics` was deprecated and will be removed in 2.0. Import `StructuralSimilarityIndexMeasure` from `torchmetrics.image` instead.
  _future_warning(
/usr/local/lib/python3.10/dist-packages/torchmetrics/utilities/prints.py:62: FutureWarning: Importing `PeakSignalNoiseRatio` from `torchmetrics` was deprecated and will be removed in 2.0. Import `PeakSignalNoiseRatio` from `torchmetrics.image` instead.
  _future_warning(


Train_Stage1 MSE: 0.0021, SSIM: 0.8419, PSNR: 26.8258
Val_Stage1 MSE: 0.0016, SSIM: 0.8380, PSNR: 28.0450
Test_Stage1 MSE: 0.0018, SSIM: 0.8062, PSNR: 27.4101
Final fine-tuned model saved as edsr_finetuned_stage1.pth


In [31]:
# Stage 2: Add Data Augmentation
# - Add augmentation (flips, rotations) to improve generalization to real data's variability.
# - Reset to Task 1 model to isolate the effect of augmentation.
# - Expected: PSNR ~29 dB, SSIM ~0.81 - slight improvement due to better generalization.
model = EDSR(scale_factor=2, num_blocks=8, channels=64)
model.load_state_dict(torch.load("/kaggle/input/edsr-best-pth/edsr_best.pth", weights_only=True))
model = finetune(model, train_loader_aug, val_loader, epochs=50, lr=5e-4, use_perceptual=False, use_radam=False, use_scheduler=False, stage_name="Stage2")
evaluate_split(model, train_loader_aug, "Train_Stage2", num_vis=3)
evaluate_split(model, val_loader, "Val_Stage2", num_vis=3)
evaluate_split(model, test_loader, "Test_Stage2", num_vis=3)
# Save final model
torch.save(model.state_dict(), "edsr_finetuned_stage2.pth")
print("Final fine-tuned model saved as edsr_finetuned_stage2.pth")


=== Stage2 ===
Model on device: cuda:0
Initial VRAM usage: 27.80 MB
LR shape: torch.Size([8, 1, 64, 64]), HR shape: torch.Size([8, 1, 128, 128])
SR shape: torch.Size([8, 1, 128, 128]), SR min: 0.0000, max: 1.0000
Epoch 1/50, Train Loss: 0.0312, Val Loss: 0.0356
Epoch 2/50, Train Loss: 0.0298, Val Loss: 0.0356
Epoch 3/50, Train Loss: 0.0299, Val Loss: 0.0356
Epoch 4/50, Train Loss: 0.0299, Val Loss: 0.0356
Epoch 5/50, Train Loss: 0.0299, Val Loss: 0.0356
Epoch 5 SR min: 0.0000, max: 0.0122
Epoch 6/50, Train Loss: 0.0299, Val Loss: 0.0356
Epoch 7/50, Train Loss: 0.0297, Val Loss: 0.0186
Epoch 8/50, Train Loss: 0.0164, Val Loss: 0.0166
Epoch 9/50, Train Loss: 0.0161, Val Loss: 0.0174
Epoch 10/50, Train Loss: 0.0159, Val Loss: 0.0182
Epoch 10 SR min: 0.0007, max: 0.1985
Epoch 11/50, Train Loss: 0.0157, Val Loss: 0.0169
Epoch 12/50, Train Loss: 0.0158, Val Loss: 0.0172
Epoch 13/50, Train Loss: 0.0158, Val Loss: 0.0194
Epoch 14/50, Train Loss: 0.0158, Val Loss: 0.0170
Epoch 15/50, Train Los

In [35]:
# Stage 3: Add Perceptual Loss (With Augmentation)
# - Continue using augmentation (train_loader_aug) to maintain generalization.
# - Add perceptual loss (VGG19, weight 0.5) to improve SSIM and push PSNR.
# - Reset to Task 1 model to isolate the effect of perceptual loss.
# - Expected: PSNR ~29.5 dB, SSIM ~0.82 - perceptual loss boosts structural fidelity.
model = EDSR(scale_factor=2, num_blocks=8, channels=64)
model.load_state_dict(torch.load("/kaggle/input/edsr-best-pth/edsr_best.pth", weights_only=True))
model = finetune(model, train_loader_aug, val_loader, epochs=50, lr=5e-4, use_perceptual=True, use_radam=False, use_scheduler=False, stage_name="Stage3")
evaluate_split(model, train_loader_aug, "Train_Stage3", num_vis=3)
evaluate_split(model, val_loader, "Val_Stage3", num_vis=3)
evaluate_split(model, test_loader, "Test_Stage3", num_vis=3)
# Save final model
torch.save(model.state_dict(), "edsr_finetuned_stage3.pth")
print("Final fine-tuned model saved as edsr_finetuned_stage3.pth")

Downloading: "https://download.pytorch.org/models/vgg19-dcbb9e9d.pth" to /root/.cache/torch/hub/checkpoints/vgg19-dcbb9e9d.pth
100%|██████████| 548M/548M [00:02<00:00, 201MB/s]  



=== Stage3 ===
Model on device: cuda:0
Initial VRAM usage: 77.21 MB
LR shape: torch.Size([8, 1, 64, 64]), HR shape: torch.Size([8, 1, 128, 128])
SR shape: torch.Size([8, 1, 128, 128]), SR min: 0.0000, max: 1.0000
Epoch 1/50, Train Loss: 0.0232, Val Loss: 0.0184
Epoch 2/50, Train Loss: 0.0163, Val Loss: 0.0176
Epoch 3/50, Train Loss: 0.0162, Val Loss: 0.0160
Epoch 4/50, Train Loss: 0.0151, Val Loss: 0.0159
Epoch 5/50, Train Loss: 0.0164, Val Loss: 0.0156
Epoch 5 SR min: 0.0000, max: 1.0000
Epoch 6/50, Train Loss: 0.0153, Val Loss: 0.0157
Epoch 7/50, Train Loss: 0.0150, Val Loss: 0.0156
Epoch 8/50, Train Loss: 0.0157, Val Loss: 0.0157
Epoch 9/50, Train Loss: 0.0157, Val Loss: 0.0158
Epoch 10/50, Train Loss: 0.0155, Val Loss: 0.0161
Epoch 10 SR min: 0.0000, max: 0.6954
Epoch 11/50, Train Loss: 0.0153, Val Loss: 0.0155
Epoch 12/50, Train Loss: 0.0153, Val Loss: 0.0158
Epoch 13/50, Train Loss: 0.0155, Val Loss: 0.0159
Epoch 14/50, Train Loss: 0.0154, Val Loss: 0.0159
Epoch 15/50, Train Los

In [38]:
# Stage 4: Optimize LR and Scheduler (With Augmentation)
# - Continue using augmentation (train_loader_aug) to maintain generalization.
# - Increase LR to 1e-3, add CosineAnnealingWarmRestarts to avoid stagnation.
# - Reset to Task 1 model to isolate the effect of LR/scheduler.
# - Expected: PSNR ~30 dB, SSIM ~0.83 - better training dynamics.
model = EDSR(scale_factor=2, num_blocks=8, channels=64)
model.load_state_dict(torch.load("/kaggle/input/edsr-best-pth/edsr_best.pth", weights_only=True))
model = finetune(model, train_loader_aug, val_loader, epochs=100, lr=1e-3, use_perceptual=True, use_radam=False, use_scheduler=True, stage_name="Stage4")
evaluate_split(model, train_loader_aug, "Train_Stage4", num_vis=3)
evaluate_split(model, val_loader, "Val_Stage4", num_vis=3)
evaluate_split(model, test_loader, "Test_Stage4", num_vis=3)
torch.save(model.state_dict(), "edsr_finetuned_stage4.pth")
print("Final fine-tuned model saved as edsr_finetuned_stage4.pth")


=== Stage4 ===
Model on device: cuda:0
Initial VRAM usage: 77.70 MB
LR shape: torch.Size([8, 1, 64, 64]), HR shape: torch.Size([8, 1, 128, 128])
SR shape: torch.Size([8, 1, 128, 128]), SR min: 0.0000, max: 1.0000
Epoch 1/100, Train Loss: 0.0233, Val Loss: 0.0224
Epoch 2/100, Train Loss: 0.0177, Val Loss: 0.0181
Epoch 3/100, Train Loss: 0.0183, Val Loss: 0.0200
Epoch 4/100, Train Loss: 0.0175, Val Loss: 0.0175
Epoch 5/100, Train Loss: 0.0167, Val Loss: 0.0173
Epoch 5 SR min: 0.0000, max: 1.0000
Epoch 6/100, Train Loss: 0.0158, Val Loss: 0.0157
Epoch 7/100, Train Loss: 0.0157, Val Loss: 0.0160
Epoch 8/100, Train Loss: 0.0150, Val Loss: 0.0156
Epoch 9/100, Train Loss: 0.0149, Val Loss: 0.0158
Epoch 10/100, Train Loss: 0.0153, Val Loss: 0.0154
Epoch 10 SR min: 0.0000, max: 1.0000
Epoch 11/100, Train Loss: 0.0147, Val Loss: 0.0152
Epoch 12/100, Train Loss: 0.0148, Val Loss: 0.0154
Epoch 13/100, Train Loss: 0.0144, Val Loss: 0.0154
Epoch 14/100, Train Loss: 0.0150, Val Loss: 0.0151
Epoch 15

In [46]:
# Stage 5: Switch to RAdam, Add Denoising, More Epochs (With Augmentation)
# - Continue using augmentation (train_loader_aug_denoise) to maintain generalization.
# - Add denoising (Gaussian blur, sigma 1.0) to handle real data noise.
# - Switch to RAdam, increase epochs to 300, bump perceptual weight to 0.7.
# - Reset to Task 1 model to isolate the effect of these changes.
# - Expected: PSNR ~30.5 dB, SSIM ~0.84 - best performance so far.
model = EDSR(scale_factor=2, num_blocks=8, channels=64)
model.load_state_dict(torch.load("/kaggle/input/edsr-best-pth/edsr_best.pth", weights_only=True))
model = finetune(model, train_loader_aug_denoise, val_loader, epochs=300, lr=1e-3, use_perceptual=True, use_radam=True, use_scheduler=True, stage_name="Stage5")
evaluate_split(model, train_loader_aug_denoise, "Train_Stage5", num_vis=3)
evaluate_split(model, val_loader, "Val_Stage5", num_vis=3)
evaluate_split(model, test_loader, "Test_Stage5", num_vis=3)
torch.save(model.state_dict(), "edsr_finetuned_stage5.pth")
print("Final fine-tuned model saved as edsr_finetuned_stage5.pth")


=== Stage5 ===
Model on device: cuda:0
Initial VRAM usage: 77.69 MB
LR shape: torch.Size([8, 1, 64, 64]), HR shape: torch.Size([8, 1, 128, 128])
SR shape: torch.Size([8, 1, 128, 128]), SR min: 0.0000, max: 0.8985
Epoch 1/300, Train Loss: 0.0397, Val Loss: 0.0216
Epoch 2/300, Train Loss: 0.0196, Val Loss: 0.0236
Epoch 3/300, Train Loss: 0.0182, Val Loss: 0.0199
Epoch 4/300, Train Loss: 0.0164, Val Loss: 0.0207
Epoch 5/300, Train Loss: 0.0155, Val Loss: 0.0191
Epoch 5 SR min: 0.0000, max: 1.0000
Epoch 6/300, Train Loss: 0.0151, Val Loss: 0.0204
Epoch 7/300, Train Loss: 0.0146, Val Loss: 0.0170
Epoch 8/300, Train Loss: 0.0144, Val Loss: 0.0171
Epoch 9/300, Train Loss: 0.0141, Val Loss: 0.0163
Epoch 10/300, Train Loss: 0.0147, Val Loss: 0.0169
Epoch 10 SR min: 0.0000, max: 1.0000
Epoch 11/300, Train Loss: 0.0146, Val Loss: 0.0165
Epoch 12/300, Train Loss: 0.0140, Val Loss: 0.0166
Epoch 13/300, Train Loss: 0.0140, Val Loss: 0.0163
Epoch 14/300, Train Loss: 0.0145, Val Loss: 0.0163
Epoch 15

In [49]:
best_model = EDSR(scale_factor=2, num_blocks=8, channels=64)
best_model.load_state_dict(torch.load("/kaggle/input/edsr_finetuned_best_stage1/pytorch/default/1/edsr_finetuned_best_Stage1.pth", weights_only=True))
evaluate_split(best_model, train_loader_aug, "Train_Stage1", num_vis=3)
evaluate_split(best_model, val_loader, "Val_Stage1", num_vis=3)
evaluate_split(best_model, test_loader, "Test_Stage1", num_vis=3)

Train_Stage1 MSE: 0.0017, SSIM: 0.8576, PSNR: 27.6013
Val_Stage1 MSE: 0.0013, SSIM: 0.8381, PSNR: 28.7617
Test_Stage1 MSE: 0.0016, SSIM: 0.8064, PSNR: 27.9314


In [50]:
best_model = EDSR(scale_factor=2, num_blocks=8, channels=64)
best_model.load_state_dict(torch.load("/kaggle/input/edsr_finetuned_best_stage2/pytorch/default/1/edsr_finetuned_best_Stage2.pth", weights_only=True))
evaluate_split(best_model, train_loader_aug, "Train_Stage2", num_vis=3)
evaluate_split(best_model, val_loader, "Val_Stage2", num_vis=3)
evaluate_split(best_model, test_loader, "Test_Stage2", num_vis=3)

Train_Stage2 MSE: 0.0016, SSIM: 0.8548, PSNR: 27.8580
Val_Stage2 MSE: 0.0012, SSIM: 0.8324, PSNR: 29.0457
Test_Stage2 MSE: 0.0015, SSIM: 0.8028, PSNR: 28.1744


In [51]:
best_model = EDSR(scale_factor=2, num_blocks=8, channels=64)
best_model.load_state_dict(torch.load("/kaggle/input/edsr_finetuned_best_stage3/pytorch/default/1/edsr_finetuned_best_Stage3.pth", weights_only=True))
evaluate_split(best_model, train_loader_aug, "Train_Stage3", num_vis=3)
evaluate_split(best_model, val_loader, "Val_Stage3", num_vis=3)
evaluate_split(best_model, test_loader, "Test_Stage3", num_vis=3)

Train_Stage3 MSE: 0.0011, SSIM: 0.8572, PSNR: 29.4790
Val_Stage3 MSE: 0.0007, SSIM: 0.8296, PSNR: 31.6401
Test_Stage3 MSE: 0.0010, SSIM: 0.8027, PSNR: 29.7925


In [53]:
best_model = EDSR(scale_factor=2, num_blocks=8, channels=64)
best_model.load_state_dict(torch.load("/kaggle/input/edsr_finetuned_best_stage4/pytorch/default/1/edsr_finetuned_best_Stage4.pth", weights_only=True))
evaluate_split(best_model, train_loader_aug, "Train_Stage4", num_vis=3)
evaluate_split(best_model, val_loader, "Val_Stage4", num_vis=3)
evaluate_split(best_model, test_loader, "Test_Stage4", num_vis=3)

Train_Stage4 MSE: 0.0012, SSIM: 0.8641, PSNR: 29.2611
Val_Stage4 MSE: 0.0007, SSIM: 0.8324, PSNR: 31.5415
Test_Stage4 MSE: 0.0011, SSIM: 0.8064, PSNR: 29.7422


In [54]:
best_model = EDSR(scale_factor=2, num_blocks=8, channels=64)
best_model.load_state_dict(torch.load("/kaggle/input/edsr_finetuned_best_stage5/pytorch/default/1/edsr_finetuned_best_Stage5.pth", weights_only=True))
evaluate_split(best_model, train_loader_aug, "Train_Stage5", num_vis=3)
evaluate_split(best_model, val_loader, "Val_Stage5", num_vis=3)
evaluate_split(best_model, test_loader, "Test_Stage5", num_vis=3)

Train_Stage5 MSE: 0.0012, SSIM: 0.8505, PSNR: 29.2129
Val_Stage5 MSE: 0.0008, SSIM: 0.8050, PSNR: 31.0698
Test_Stage5 MSE: 0.0011, SSIM: 0.7889, PSNR: 29.4955
